In [1]:
from b_field import structured_field as s
import numpy as np
import numpy.ma as ma
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.integrate import simps
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.collections import LineCollection
from matplotlib import cm
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.ticker import MaxNLocator
from gammaALPs.nel.icm import NelICM
from gammaALPs.bfields.gauss import Bgaussian
from gammaALPs.core import ModuleList, ALP,   Source
from scipy.spatial.transform import Rotation as R_
import time
from iminuit import Minuit

In [2]:
def cart_to_sphere(x2, y2, z2):
    r2 = np.sqrt(x2**2 + y2**2 + z2**2)
    theta2 = np.arccos(z2 / r2)
    phi2 = np.arctan2(y2, x2)
    return r2, theta2, phi2

def sphere_to_cart(r, theta, phi):
    x1 = r * np.sin(theta) * np.cos(phi)
    y1 = r * np.sin(theta) * np.sin(phi)
    z1 = r * np.cos(theta)
    return x1, y1, z1


alpha = 5.7634591968
c = 1
F_0 = c * (alpha * np.cos(alpha) - np.sin(alpha)) * alpha**2
norm_ = np.sqrt((3 * F_0 + alpha**5)**2) * 2 / (3 * alpha**2)

def trafo(r, theta, phi):
    '''Returns trafo used for vector fields, i.e. Field A cartesian = S * Field A spherical. S is returned'''
    mat = np.array([[np.sin(theta) * np.cos(phi), np.cos(theta) * np.cos(phi), -np.sin(phi)],
                    [np.sin(theta) * np.sin(phi), np.cos(theta) * np.sin(phi), np.cos(phi)],
                    [np.cos(theta), -np.sin(theta), 0]])
        #outdata[c, :, :] = mat[:, :]
    
    return mat


def rotation(n, x, angle):
    '''Rotation of vector x around n by angle.'''
    return n * np.inner(n, x) + np.cos(angle) * np.cross(np.cross(n, x), n) + np.sin(angle) * np.cross(n, x)

def cart_rotation(alpha, beta):
    '''Returns rotation matrices in cartesian coordinates x, y, z.'''
    rot_x = np.array([[1,             0,              0],
                      [0, np.cos(alpha), -np.sin(alpha)],
                      [0, np.sin(alpha),  np.cos(alpha)]])
    rot_y = np.array([[ np.cos(beta), 0, np.sin(beta)],
                      [            0, 1,            0],
                      [-np.sin(beta), 0, np.cos(beta)]])
    rot_z = np.array([[np.cos(beta), -np.sin(beta), 0],
                      [np.sin(beta),  np.cos(beta), 0],
                      [           0,             0, 1]])
    return rot_x, rot, y, rot_z

def b_r(r, theta):
    
    zero_val = - np.cos(theta) * (6 * F_0 + 2 * alpha**5 * c) / (3 * alpha**2)
    if np.isclose(r, 0):
        val = zero_val
    else:
        val = 2 * np.cos(theta) * f(r) / r**2
    if r > 1:
        val = 0
    return 1 / norm_ * val
    

def b_theta(r, theta):
    zero_val = np.sin(theta) * (6 * F_0 + 2 * alpha**5 * c) / (3 * alpha**2)
    if np.isclose(r, 0):
        val = zero_val
    else:
        val = - np.sin(theta) * f_prime(r) / r
    if r > 1:
        val = 0
    return 1 / norm_ * val


def b_phi(r, theta):
    zero_val = 0
    if np.isclose(r, 0):
        val = zero_val
    else:
        val = alpha * np.sin(theta) * f(r) / r
    if r > 1:
        val = 0
    return 1 / norm_ * val


def f(r):
    return c * (alpha * np.cos(alpha * r) - np.sin(alpha * r) / r) \
           - F_0 * r**2 / alpha**2


def f_prime(r):
    return c * ( - alpha**2 * np.sin(alpha * r) \
                - alpha * np.cos(alpha * r) / r \
                + np.sin(alpha * r) / r**2) \
               - 2 * F_0 * r / alpha**2 

def rotation_measure(z, b_par, nel):
    #print(z)
    #print(b_par)
    #print(nel)
    return 812 * simps(b_par * nel, z)

def projection(field, direction):
    return np.inner(field, direction)

def axis_helper(x):
    l = x.shape[0]
    dx = x[1] - x[0]
    return np.linspace(x[0] - dx / 2, x[-1] + dx / 2, num=l + 1)


def B(B0, x1, x2, x3, in_sys, out_sys):
    #get input coordinates
    if in_sys == 'cart':
        r, theta, phi = cart_to_sphere(x1, x2, x3)
    elif in_sys=='sph':
        r, theta, phi = x1, x2, x3
    else:
        print('wrong coords')
    
    #calculate bfield
    B = np.array([b_r(r / R, theta), b_theta(r / R, theta), b_phi(r / R, theta)])
    #print(B.shape)
    #get output coordinates
    if out_sys == 'sph':
        return B * B0
    elif out_sys == 'cart':
        return B0 * np.matmul(trafo(r, theta, phi), B.transpose()).transpose()

#def kpc_to_arcmin()

def B_rot(B, rotvec):
    return np.matmul(rotvec.as_matrix(), B.transpose()).transpose()

In [3]:
nel = NelICM(n0 = 39., n2 = 4.05, r_abell = 500., r_core = 80., r_core2 = 280., beta = 1.2, beta2= 0.58, eta = 0.5)
def generate_B_struc(xx, yy, zz, B0, R, theta, phi):

    theta_ = np.radians(theta)
    phi_ = np.radians(phi)
    
    #get rotation
    #incl = R_.from_rotvec(theta_ * np.array([1, 0, 0]))    # tilts symmetry axis around x-axis
    pa = R_.from_rotvec(phi_ * np.array([0, 0, 1]))        # rotates around z axis, position angle phi
    
    incl = R_.from_rotvec(theta_ * np.array([1, 0, 0]))
    incl = R_.from_rotvec(B_rot(incl.as_rotvec(), pa))

    
    #get magnetic field at each point
    B_r = np.zeros((xx.flatten().shape[0], 3))
    c_list = list(zip(xx.flatten(), yy.flatten(), zz.flatten()))
    for c_, (x_, y_, z_) in enumerate(c_list):
        r_, theta_, phi_ = cart_to_sphere(x_, y_, z_)
        if np.isnan(theta_):
            #check for previous and next few entries for r=0 and take their theta
            print(c_)
            _, theta_, _ = cart_to_sphere(*c_list[c_ + 1])
        b_temp = B_rot(B(B0, r_, theta_, phi_, 'sph', 'cart'), incl)
        B_r[c_] = b_temp #B_rot(b_temp, pa)
    B_r = B_r.reshape(*xx.shape, 3)
    return B_r[:, :, :, 2]


def make_rm(xx, yy, zz, b):
    global nel
    #get electron density
    
    rr = np.sqrt(xx**2 + yy**2 + zz**2)
    nel_mesh = np.where(rr<500, nel(rr)*1e-3, 0)    # limited to r<r_abell, in cm^-3
    #print(rr)
    #print(nel_mesh)
    rm = np.zeros(xx.shape[:2])
    #for i in range(rm.shape[0]):
    #    for j in range(rm.shape[1]):
    #        print(b[i, j, :] * nel_mesh[i, j, :])
    rm = 812 * simps(b * nel_mesh, x=zz, axis=-1)
    return rm

def init_gauss(B0, seed):
    alp = ALP(1, 1)
    source = Source(z = 0.017559, ra = '03h19m48.1s', dec = '+41d30m42s')
    m = ModuleList(alp, source, pin=0.5 * np.diag((1, 1, 0)), EGeV=np.linspace(2, 2.6, num=10))
    m.add_propagation("ICMGaussTurb", 
                      0,
                      nsim = 1,
                      B0 = B0,
                      n0 = 39.,
                      n2 = 4.05,
                      r_abell = 500.,
                      r_core = 80.,
                      r_core2 = 280.,
                      beta = 1.2,
                      beta2= 0.58,
                      eta = 0.5,
                      kL = 2 * np.pi / 35, 
                      kH = 2 * np.pi / 0.7,
                      q = -2.80,
                      seed = seed
                      )
    #m.add_propagation("EBL",1, model = 'dominguez')
    #m.add_propagation("GMF",2, model = 'jansson12', model_sum = 'ASS')
    return m.modules[0]._Bfield_model

def generate_B_random(Bg, num, z):
    B, psi = Bg.new_Bn(z, nsim=num)
    B_ = np.zeros((num, z.size))
    for c_, v_ in enumerate(B):
        B_[c_] = v_ * np.sin(psi[c_])
    return B_



def make_random_rm(x, y, z, B):
    # x, y are single coordinates, z is vector along which B is calculated
    r = np.sqrt(x**2 + y**2 + z**2)
    #scale B field, fixed eta to 0.5
    B_scaled = B * np.power(nel(r) / nel(0), 0.5)
    
    rm = 812 * simps(B_scaled * np.where(r < 500, nel(r) * 1e-3, 0), z)
    return rm

def make_rm_dist(x, y, B0, rm_num, z_steps, seed):
    Bgaus = init_gauss(B0, seed)
    R = 500
    dL = R / z_steps
    z = np.linspace(dL / 2, R - dL / 2, num=z_steps, endpoint=True)
    Bfield = generate_B_random(Bgaus, rm_num, z)
    rm = np.zeros(rm_num)
    #rm_ = []
    for c_, B in enumerate(Bfield):
        rm[c_] = make_random_rm(x, y, z, B)
        #rm_.append(make_random_rm(x, y, z, B))
    return rm
    print(rm)
    pars = norm.fit(rm[1:])
    return pars[1]

In [4]:
start = time.time()

rm = make_rm_dist(0, 0, 10, 1000, 5000, 530)
#print(rm)
#print(norm.fit(rm))
end = time.time()
print('elapsed time', end - start)

KeyboardInterrupt: 

In [ ]:
for i in np.linspace(100, 1000, num=10, endpoint=True, dtype=int):
    pars = norm.fit(rm[:i-1])
    print(pars)
#plt.plot(x, sigma)
#plt.xscale('log')
#plt.yscale('log')

In [ ]:
np.savetxt('rm.txt', rm, header='0, 0, 10, 1000, 5000, 530')

In [ ]:
rm = np.loadtxt('rm.txt')
rm *= np.sqrt(2)

In [ ]:
# can use this in princible to estimate uncertainty of MLE
# use minuit for fitting, norm.nnlf() as cost function, set that option to 0.5 in minuit
x0, sigma = norm.fit(rm)
print(x0, sigma)
print(norm.nnlf((x0, sigma), rm))
def cost(pos, rms):
    global rm
    return norm.nnlf((pos, rms), rm)
m = Minuit(cost, pos=x0, rms=sigma, errordef=0.5)
migrad = m.migrad()
minos = m.minos()
#norm.nnlf((x0, sigma), rm)

In [ ]:
for i in range(2):
    print(m.merrors[i].lower, m.merrors[i].upper)

In [ ]:
m.values[1]

In [ ]:
np.savetxt('sigma_first_run.txt', sigma)
np.savetxt('r_first_run.txt', x)

In [ ]:
x = np.linspace(0, 500, num=50)
sigma = np.zeros(x.shape)
for c_, x_ in enumerate(x):
    sigma[c_] = make_rm_dist(x_, 0, 10, 1000, 5000, None)
    break

In [ ]:
nums = [100, 1000, 2000, 5000, 6000, 8000, 10000]
for i in nums:
    print(make_rm_dist(200, 0, 10, 2, i, 69))
    
#should use ~5000 points along LOS

In [ ]:

# use this snippet to fill up rm map, take x, y from some meshgrid.


x = 0
y = 0

Bgaus = init_gauss(10)
dL = 1
R = 500
z = np.linspace(dL / 2, R - dL / 2, num=200, endpoint=True)
Bfield = generate_B_random(Bgaus, 10, z)
#for b in Bfield:
#    plt.plot(z, b)
#for B in Bfield:
#    plt.plot(z, B * np.power(nel(z) / nel(0), 0.5))
for B in Bfield:
    print(make_random_rm(x, y, z, B))

In [ ]:
R = 93
B0 = 8.3
theta = 0
phi = 0

x = np.linspace(-R, R, num=10)
y = np.linspace(-R, R, num=10)
z = np.linspace(0, R, num=10)
xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
B_los = generate_B_struc(xx, yy, zz, B0, R, theta, phi)
rm = make_rm(xx, yy, zz, B_los)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.pcolor()

In [ ]:
#copied from probs.py
def module(self):
    m = ModuleList(self.alp, self.source, pin=self.pin, EGeV=self.EGeV)
    m.add_propagation("ICMGaussTurb", 
                      0,
                      nsim = self.nsim,
                      B0 = self.B0,
                      n0 = 39.,
                      n2 = 4.05,
                      r_abell = 500.,
                      r_core = 80.,
                      r_core2 = 280.,
                      beta = 1.2,
                      beta2= 0.58,
                      eta = 0.5,
                      kL = self.kL, 
                      kH = self.kH,
                      q = self.q,
                      seed = self.seed
                      )
    m.add_propagation("EBL",1, model = 'dominguez')
    m.add_propagation("GMF",2, model = 'jansson12', model_sum = 'ASS')
    return m

In [ ]:
r = np.linspace(200, 600, num=20)
np.where(r < 500, nel(r) * 1e-3, 0)

In [ ]:
norm.fit(rm)